### Imports

In [1]:
import sys
import os

project_path = os.path.abspath(os.path.join('../'))
if project_path not in sys.path:
    sys.path.append(project_path)

from utils.load_data import load_fer2013, parse_fer2013, \
                            load_CKPlus, fer_csv_to_png
from utils.preprocess import resnet_preprocess
from utils.generators import init_resnet_generator, init_cnn_generator
from utils.plots import plot_model_history
from models.build import build_resnet_model, build_cnn_baseline_model

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

import numpy as np

Using TensorFlow backend.


### Loading datasets

In [2]:
def unpack_csv(data_dir='../data/fer', file_path='../data/fer2013/fer2013.csv'):
    if not os.path.exists(data_dir):
        os.mkdir(data_dir)
        os.mkdir(data_dir + '/train')
        os.mkdir(data_dir + '/val')
        os.mkdir(data_dir + '/test')
        fer_csv_to_png(file_path=file_path,
                       data_dir=data_dir)

# Local
unpack_csv(data_dir='../data/fer', file_path='../data/fer2013/fer2013.csv')
# Google colab
# unpack_csv(data_dir='/fer', file_path='/content/drive/MyDrive/fer2013.csv')

train_data, val_data, test_data = load_fer2013('../data/fer2013/fer2013.csv')
X_train, Y_train = parse_fer2013(train_data)
X_val, Y_val = parse_fer2013(val_data)
X_test, Y_test = parse_fer2013(test_data)
X_test = resnet_preprocess(X_test, target_size=(197, 197), target_channels=3)

### Model training parameters

In [3]:
num_train = 28709
num_val = 3589
batch_size = 128
num_epochs = 150
patience = 20
saved_models_path = '../models/saved_models/'
dataset_name = 'fer2013'
model_name = '_resnet50_1'
history_name = '_hist' + model_name

# ../models/saved_models/fer2013_resnet50_1
model_path = saved_models_path + dataset_name + model_name

# ../models/saved_models/fer2013_hist_resnet50_1
history_path = saved_models_path + dataset_name + history_name

### Data generators

In [4]:
train_gen = init_resnet_generator(data_dir='../data/fer/train', with_aug=True)
val_gen = init_resnet_generator(data_dir='../data/fer/val', with_aug=False)

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


### Callbacks

In [5]:
cpt_path = model_path + '.hdf5'
early_stop = EarlyStopping('val_loss', patience=patience)
model_checkpoint = ModelCheckpoint(cpt_path, save_best_only=True)
callbacks = [model_checkpoint, early_stop]

In [6]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.mixed_precision import experimental as mixed_precision

In [7]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3090, compute capability 8.6
Instructions for updating:
Use tf.keras.mixed_precision.LossScaleOptimizer instead. LossScaleOptimizer now has all the functionality of DynamicLossScale
Compute dtype: float16
Variable dtype: float32


In [8]:
# tf.keras.mixed_precision.set_global_policy('mixed_float16')
# tf.keras.mixed_precision.global_policy()
import tensorflow

### Build and compile model

In [9]:
model = build_resnet_model()
opt = tensorflow.keras.optimizers.Adam(0.001)
opt_mixed_precision = keras.mixed_precision.LossScaleOptimizer(opt)
model.compile(optimizer=opt_mixed_precision, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vggface_resnet50 (Functional (None, 1, 1, 2048)        23561152  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 4096)              8392704   
_________________________________________________________________
dropout_1 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              4195328   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0

### Training

In [10]:
history = model.fit_generator(train_gen,
                    validation_data=val_gen,
                    epochs=num_epochs,
                    steps_per_epoch=len(X_train)//batch_size,
                    validation_steps=len(X_val)//batch_size,
                    callbacks=callbacks, workers=10, verbose=1, max_queue_size=100)

# Saving final model and history
val_acc = history.history['val_accuracy'][-1]
cnt_epoch = len(history.history['val_accuracy'])
np.save(f'{history_path}-e{cnt_epoch:02d}-a{val_acc:.2f}.npy', history.history)
model.save(f'{model_path}-e{cnt_epoch:02d}-a{val_acc:.2f}.hdf5')

/home/ubuntu/anaconda3/envs/Tensorflow-gpu/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/150
224/224 [==============================] - 102s 291ms/step - loss: 3.7218 - accuracy: 0.3133 - val_loss: 1.9140 - val_accuracy: 0.1691
Epoch 2/150
224/224 [==============================] - 51s 219ms/step - loss: 1.2266 - accuracy: 0.5389 - val_loss: 1.9690 - val_accuracy: 0.1702
Epoch 3/150
224/224 [==============================] - 49s 211ms/step - loss: 1.1192 - accuracy: 0.5900 - val_loss: 1.3792 - val_accuracy: 0.4665
Epoch 4/150
224/224 [==============================] - 50s 216ms/step - loss: 1.0510 - accuracy: 0.6094 - val_loss: 0.9923 - val_accuracy: 0.6253
Epoch 5/150
224/224 [==============================] - 49s 211ms/step - loss: 1.0126 - accuracy: 0.6184 - val_loss: 0.9500 - val_accuracy: 0.6443
Epoch 6/150
224/224 [==============================] - 49s 211ms/step - loss: 0.9899 - accuracy: 0.6296 - val_loss: 0.9506 - val_accuracy: 0.6445
Epoch 7/150
224/224 [==============================] - 51s 221ms/step - loss: 0.9560 - accuracy: 0.6405 - val_loss: 0.9479 

### Loading (if needed)

In [11]:
# Uncomment to load
# history = np.load(history_path + '-e03-a0.41.npy', allow_pickle=True)
# history = history.item()
# model = load_model(model_path + '-e41-a0.71.hdf5')

### Evaluation

In [12]:
loss, acc = model.evaluate(X_test, Y_test)
print("Model accuracy: {:5.2f}%".format(100 * acc))

113/113 [==============================] - 31s 143ms/step - loss: 0.8113 - accuracy: 0.7236
Model accuracy: 72.36%


In [ ]:
print(histroy)

### Ploting curves

In [13]:
plot_model_history(history)

TypeError: 'History' object is not subscriptable